# Object Detection - SSD Mobilenet V2 from TensorFlow Hub (Base64 Images)

This model identifies objects present in images, returning its scores, labels and bounding boxes.

We are using a dataset from [UCF](https://www.crcv.ucf.edu/data/GMCP_Geolocalization/#Dataset) and the model [SSD Mobilenet V2](https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1) from [TensorFlow Hub](https://www.tensorflow.org/hub).

<a href="https://colab.research.google.com/github/VertaAI/examples/blob/main/deployment/tfhub/object-detection/base64/object-detection-base64-predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

This notebook has been tested with **Python 3.8.15** and the following package versions:

In [ ]:
%%capture
!pip install numpy==1.23.5
!pip install verta==0.21.1

## Imports

In [ ]:
import base64
import concurrent.futures
import numpy as np
import os
import requests
import time
import json

from verta import Client

## Verta Set Up

In [ ]:
os.environ['VERTA_HOST'] = 'app.verta.ai'
os.environ['VERTA_EMAIL'] = 'caio@verta.ai'
os.environ['VERTA_DEV_KEY'] = '68b041eb-553f-4009-acb6-51a533755324'

In [ ]:
client = Client(os.environ['VERTA_HOST'] , debug=True)

In [ ]:
endpoint = client.get_or_create_endpoint('object-detection-base64')

In [ ]:
model = endpoint.get_deployed_model()

## Show Process Metrics

In [ ]:
def show_metrics(images, start_time, end_time):
    total_time = end_time - start_time
    total_time = time.strftime('%Hh %Mm %Ss', time.gmtime(total_time))
    
    print(f"Processing Time: {total_time} for {images} images.")

## Predict

In [ ]:
def process_image(image):
    with open(f"images/{image}", 'rb') as img:
        img_bytes = base64.b64encode(img.read())
        img_str = img_bytes.decode('utf-8')
        img_str = json.dumps(img_str)
    
    return model.predict([image, img_str])

In [ ]:
# n_imgs = 100
# n_imgs = 1000
n_imgs = 10000

In [ ]:
images = os.listdir('images')

In [ ]:
images = images[:n_imgs]

In [ ]:
results = []
start_time = time.time()

with concurrent.futures.ThreadPoolExecutor() as executor:
    for image, result in zip(images, executor.map(process_image, images)):
        results.append([image, result])

end_time = time.time()

In [ ]:
show_metrics(n_imgs, start_time, end_time)